# 1. Import Libraries

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import requests
from io import BytesIO
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, EfficientNetB3, DenseNet169, Xception, ConvNeXtBase
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

SEED = 244
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 2. Load Dataset

### 2.1 Define GitHub Repo & Folder Path

In [2]:
GITHUB_REPO = "prattapong/Commercial-Airplane-Model-Image-Classification"
GITHUB_FOLDER = "images"
GITHUB_API_URL = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{GITHUB_FOLDER}"

### 2.2 Fetch Image URLs and Load Image

In [3]:
# Fetch Image URLs Automatically
def get_image_urls():
    response = requests.get(GITHUB_API_URL)
    if response.status_code != 200:
        raise Exception(f"Error fetching images: {response.json()}")

    image_urls = {}
    for folder in response.json():
        if folder["type"] == "dir":  # Ensure it's a folder (A350, B787, A320)
            class_name = folder["name"]
            image_urls[class_name] = []
            folder_url = folder["url"]

            # Fetch image files in each class folder
            folder_response = requests.get(folder_url)
            if folder_response.status_code == 200:
                for file in folder_response.json():
                    if file["name"].lower().endswith((".jpg", ".jpeg", ".png")):
                        image_urls[class_name].append(file["download_url"])

    return image_urls

# Load Images Using Image.open()
def load_images(image_urls):
    IMG_SIZE = (224, 224)  # Resize all images to 224x224
    X, y = [], []

    total_images = sum(len(urls) for urls in image_urls.values())  # Total number of images
    progress_bar = tqdm(total=total_images, desc="Loading Images", unit="img")

    for label, urls in image_urls.items():
        for url in urls:
            try:
                response = requests.get(url)
                img = Image.open(BytesIO(response.content)).convert("RGB")  # Load image
                img = img.resize(IMG_SIZE)
                X.append(np.array(img) / 255.0)  # Normalize
                y.append(label)
            except Exception as e:
                print(f"Error loading {url}: {e}")
            progress_bar.update(1)  # Update progress bar

    progress_bar.close()
    return np.array(X), pd.Categorical(y).codes

In [ ]:
image_urls = get_image_urls()
X, y = load_images(image_urls)

Loading Images:   1%|          | 6/605 [00:00<00:44, 13.33img/s]

Error loading https://raw.githubusercontent.com/prattapong/Commercial-Airplane-Model-Image-Classification/main/images/Airbus%20A320/1.jpg: cannot identify image file <_io.BytesIO object at 0x7fedf99c93f0>


Loading Images:   3%|▎         | 18/605 [00:01<00:49, 11.80img/s]

Error loading https://raw.githubusercontent.com/prattapong/Commercial-Airplane-Model-Image-Classification/main/images/Airbus%20A320/1180206_1719464150.jpg: cannot identify image file <_io.BytesIO object at 0x7fee25a2ed90>


Loading Images:   3%|▎         | 20/605 [00:02<00:58,  9.96img/s]

...

Loading Images:  99%|█████████▊| 597/605 [01:19<00:00,  8.77img/s]

Error loading https://raw.githubusercontent.com/prattapong/Commercial-Airplane-Model-Image-Classification/main/images/Boeing%20787/Screenshot%202025-03-08%20202549.JPG: cannot identify image file <_io.BytesIO object at 0x7fedf9ed4d10>
Error loading https://raw.githubusercontent.com/prattapong/Commercial-Airplane-Model-Image-Classification/main/images/Boeing%20787/Screenshot%202025-03-08%20202639.JPG: cannot identify image file <_io.BytesIO object at 0x7fedfbf9f150>
Error loading https://raw.githubusercontent.com/prattapong/Commercial-Airplane-Model-Image-Classification/main/images/Boeing%20787/Screenshot%202025-03-08%20202741.JPG: cannot identify image file <_io.BytesIO object at 0x7fee25a2ee30>


Loading Images:  99%|█████████▉| 599/605 [01:20<00:00, 10.44img/s]

Error loading https://raw.githubusercontent.com/prattapong/Commercial-Airplane-Model-Image-Classification/main/images/Boeing%20787/Screenshot%202025-03-08%20203012.JPG: cannot identify image file <_io.BytesIO object at 0x7fee25a2e070>


Loading Images:  99%|█████████▉| 601/605 [01:20<00:00,  5.35img/s]

# 3. Data-preprocessing

In [5]:
def augment_data(X_train):
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest"
    )

    datagen.fit(X_train)

    return datagen

# 4. Train model

### 4.1 List 5 pre-trained models

In [6]:
MODELS = {
    "ResNet50": ResNet50,
    "VGG16": VGG16,
    "DenseNet169": DenseNet169,
    "Xception": Xception,
    "ConvNeXtBase": ConvNeXtBase
}

### 4.2 Initiate training loop

In [11]:
def train_and_evaluate_model(model_name, X, y, batch_size, epochs, learning_rate, dropout_rate, num_unfreezed_layer=0):
    print(f'*** Training {model_name} ***')
    print(f'- Batch Size: {batch_size}')
    print(f'- LR: {learning_rate}')
    print(f'- Dropout: {dropout_rate}')
    print(f'- Last "{num_unfreezed_layer}" layers unfreezed\n')

    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=SEED
    )

    # Define Input Layer
    input_tensor = Input(shape=(224, 224, 3))

    # Load Pretrained Model
    base_model = MODELS[model_name](weights="imagenet", include_top=False, input_tensor=input_tensor)

    # Unfreeze Layers If Needed
    if num_unfreezed_layer == 0:
        for layer in base_model.layers:
            layer.trainable = False
    else:
        for layer in base_model.layers[-num_unfreezed_layer:]:
            layer.trainable = True

    # Add Custom Classification Head
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    output_layer = Dense(len(np.unique(y)), activation="softmax")(x)

    # Build Model
    model = Model(inputs=input_tensor, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    # Data Augmentation
    datagen = augment_data(X_train)

    # Model Checkpointing
    model_path = f'best_{model_name}.keras'  # Use .keras format
    checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

    # Train Model
    history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                        validation_data=(X_test, y_test),
                        epochs=epochs,
                        callbacks=[checkpoint])

    # Load Best Model (Fix for ConvNeXtBase)
    custom_objects = get_custom_objects() if model_name == "ConvNeXtBase" else None
    best_model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)

    # Evaluate on Test Data
    test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
    print(f"\n{model_name} Test Accuracy: {test_accuracy:.4f}")

    return model_name, test_accuracy, history

def compare_models(X, y, learning_rates, dropout_rates, batch_sizes, num_unfreezed_layers, epochs=50):
    results = []

    # Loop through all model hyperparameters with tqdm progress bar
    for model_name in MODELS.keys():
        for lr in learning_rates:
            for dr in dropout_rates:
                for batch_size in batch_sizes:
                    for num_unfreezed_layer in num_unfreezed_layers:
                        result = train_and_evaluate_model(model_name, X, y, batch_size, epochs, lr, dr, num_unfreezed_layer)
                        results.append(result)
                        print('------------------------------------------------------------------------\n')

    # Sort models by accuracy
    results.sort(key=lambda x: x[1], reverse=True)
    print("\nModel Performance Ranking:")
    for rank, (model_name, accuracy, _) in enumerate(results, 1):
        print(f"{rank}. {model_name}: {accuracy:.4f}")

    return results

In [8]:
# image_urls = get_image_urls()
# X, y = load_images(image_urls)

### 4.3 Set Hyper-parameter

In [9]:
num_unfreezed_layers = [0]
learning_rates = [0.0001]
dropout_rates = [0.2]
batch_sizes = [8]

### 4.4 Train all models and parameters

In [ ]:
best_model = compare_models(
    X = X,
    y = y,
    epochs = 5,

    ############## Hyper-parameter ##############
    learning_rates = learning_rates,
    dropout_rates = dropout_rates,
    batch_sizes = batch_sizes,
    num_unfreezed_layers = num_unfreezed_layers
)

*** Training ResNet50 ***
- Batch Size: 8
- LR: 0.0001
- Dropout: 0.2
- Last "0" layers unfreezed

Epoch 1/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.2970 - loss: 1.1833
Epoch 1: val_accuracy improved from -inf to 0.39669, saving model to best_ResNet50.keras
61/61 ━━━━━━━━━━━━━━━━━━━━ 44s 547ms/step - accuracy: 0.2973 - loss: 1.1829 - val_accuracy: 0.3967 - val_loss: 1.0915
Epoch 2/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.3739 - loss: 1.1123
Epoch 2: val_accuracy did not improve from 0.39669
61/61 ━━━━━━━━━━━━━━━━━━━━ 12s 124ms/step - accuracy: 0.3738 - loss: 1.1126 - val_accuracy: 0.3471 - val_loss: 1.0889
Epoch 3/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.2845 - loss: 1.1514
Epoch 3: val_accuracy did not improve from 0.39669
61/61 ━━━━━━━━━━━━━━━━━━━━ 6s 102ms/step - accuracy: 0.2849 - loss: 1.1515 - val_accuracy: 0.3223 - val_loss: 1.1002
Epoch 4/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.3010 - loss: 1.1630
Epoch 4: val_accur

# 5. Show result of a sample

In [21]:
def compute_gradcam(model, img_array, layer_name):
    """
    Computes Grad-CAM heatmap for a given image and model.

    Parameters:
    - model: Trained Keras model
    - img_array: Preprocessed image array with shape (1, height, width, channels)
    - layer_name: The name of the convolutional layer to extract gradients from

    Returns:
    - heatmap: Grad-CAM heatmap (numpy array)
    """
    grad_model = Model(inputs=model.input, outputs=[model.get_layer(layer_name).output, model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = tf.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    for i in range(conv_outputs.shape[-1]):
        conv_outputs[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0)  # ReLU
    heatmap /= np.max(heatmap)  # Normalize
    return heatmap

def overlay_heatmap(img, heatmap, alpha=0.5):
    """
    Overlays the Grad-CAM heatmap on the original image.

    Parameters:
    - img: Original image
    - heatmap: Grad-CAM heatmap
    - alpha: Transparency factor (default=0.5)

    Returns:
    - Superimposed image with heatmap
    """
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    return superimposed_img

def show_prediction_with_gradcam(model, X, y, index, class_labels, layer_name="convnext_base_stage4_block2_layer_scale"):
    """
    Displays an image with the model's prediction, actual label, and Grad-CAM visualization.

    Parameters:
    - model: Trained Keras model
    - X: NumPy array of images
    - y: NumPy array of actual labels
    - index: Index of the image to display
    - class_labels: List of class names corresponding to label indices
    - layer_name: Name of the convolutional layer to compute Grad-CAM from (default: last ConvNeXt block)
    """
    sample_image = np.array(X[index])
    sample_input = np.expand_dims(sample_image, axis=0)

    # Predict
    predictions = model.predict(sample_input)
    predicted_class = np.argmax(predictions)
    actual_class = y[index]

    predicted_label = class_labels[predicted_class]
    actual_label = class_labels[actual_class]

    # Compute Grad-CAM
    heatmap = compute_gradcam(model, sample_input, layer_name)
    superimposed_img = overlay_heatmap((sample_image * 255).astype(np.uint8), heatmap)

    # Display Results
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # Original Image
    axes[0].imshow(sample_image)
    axes[0].axis("off")
    axes[0].set_title(f"Original Image\nActual: {actual_label}\nPredicted: {predicted_label}", fontsize=12, color="blue")

    # Grad-CAM Overlay
    axes[1].imshow(superimposed_img)
    axes[1].axis("off")
    axes[1].set_title("Grad-CAM Visualization", fontsize=12, color="red")

    plt.show()

In [25]:
class_labels = ["Airbus A320", "Airbus A350", "Boeing 787"]
# best_model = tf.keras.models.load_model("best_DenseNet169.h5")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'best_DenseNet169.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [23]:
for i in [0,10,402,405,500, 600]:
    show_prediction(best_model, X, y, index=i, class_labels=class_labels)

AttributeError: 'list' object has no attribute 'predict'

In [24]:
best_model

[('DenseNet169',
  0.6363636255264282,
  <keras.src.callbacks.history.History at 0x78474e749150>),
 ('Xception',
  0.4958677589893341,
  <keras.src.callbacks.history.History at 0x784605569ed0>),
 ('ConvNeXtBase',
  0.4628099203109741,
  <keras.src.callbacks.history.History at 0x78460436d1d0>),
 ('ResNet50',
  0.40495866537094116,
  <keras.src.callbacks.history.History at 0x7847c8233890>),
 ('EfficientNetB3',
  0.35537189245224,
  <keras.src.callbacks.history.History at 0x7847b41251d0>)]